In [3]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import os


In [34]:
files = os.listdir('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/projection_parasites/')

In [35]:

for i in range(len(files)):
    # Read the CSV file
    df = pd.read_csv('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/projection_parasites/' + files[i])

    # Rename columns
    df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)

    # Create the graph with weights
    G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight')

    print(f'name: {files[i]}, edges: {len(G.edges(data=True))}, nodes: {len(G.nodes)}')


name: simple_disparity_filter.csv, edges: 598944, nodes: 17426
name: pearson_noise_corrected.csv, edges: 109730, nodes: 15369
name: hyperbolic_high_salience_skeleton.csv, edges: 598944, nodes: 17426
name: resource_allocation_naive.csv, edges: 598944, nodes: 17426
name: simple_noise_corrected.csv, edges: 598944, nodes: 17426


In [40]:
resource_all_df = pd.read_csv('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/projection_parasites/' + files[2])
# Rename columns
resource_all_df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)

# Create the graph with weights
G = nx.from_pandas_edgelist(resource_all_df, source='source', target='target', edge_attr='weight')

In [45]:
threshold = 0.01

# Get all edge weights
edge_weights = nx.get_edge_attributes(G, 'weight')

# Filter edges with weights above the threshold
filtered_edges = [(u, v, w) for (u, v), w in edge_weights.items() if w >= threshold]

# Create a new graph with the filtered edges
filtered_G = nx.Graph()  # Use nx.DiGraph() if the original graph is directed
filtered_G.add_weighted_edges_from(filtered_edges)

# Find all connected components
connected_components = list(nx.connected_components(filtered_G))

# Find the largest connected component
largest_component = max(connected_components, key=len)

# Create a subgraph of the largest connected component
largest_subgraph = filtered_G.subgraph(largest_component)

# Print details
print(f"Number of nodes in largest component: {len(largest_subgraph.nodes)}")
print(f"Number of edges in largest component: {len(largest_subgraph.edges)}")


Number of nodes in largest component: 15899
Number of edges in largest component: 438649


In [ ]:
Number of nodes in largest component: 16290
Number of edges in largest component: 597342